In [17]:
import pandas as pd
import numpy as np

import sklearn.metrics   as mt
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation

from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.decomposition import PCA

https://github.com/manoelmendonca/Ensaio-de-Machine-Learning/tree/main

In [18]:
path_xtrain = 'datasets//X_dataset.csv'

X_train = pd.read_csv( path_xtrain )

# KMeans

In [19]:
from sklearn.cluster import KMeans
import pandas as pd

# Definir uma lista de números de clusters para testar
n_clusters_list = [2, 3, 4, 5, 6, 7, 8, 9, 10]

# Inicializar variáveis para armazenar o melhor resultado
best_score = float('-inf')
best_n_clusters = None
best_important_features = None

# Loop for sobre diferentes números de clusters
for n_clusters in n_clusters_list:
    # Treinar o modelo KMeans
    kmeans = KMeans(n_clusters=n_clusters)
    labels = kmeans.fit_predict(X_train)

    # Obter os centroides
    centroids = kmeans.cluster_centers_

    # Identificar as features mais importantes nos centroides
    feature_importance = centroids.max(axis=0) - centroids.min(axis=0)
    important_features = X_train.columns[feature_importance.argsort()[::-1]]

    # Calcular o score (você pode usar uma métrica interna do KMeans ou uma externa, como a silhouette_score)
    score = kmeans.score(X_train)

    # Verificar se o resultado atual é o melhor até agora
    if score > best_score:
        best_score = score
        best_n_clusters = n_clusters
        best_important_features = important_features

# Imprimir o melhor resultado
print("Melhor número de clusters:", best_n_clusters)
print("Melhores features mais importantes nos centroides:")
print(best_important_features)




Melhor número de clusters: 10
Melhores features mais importantes nos centroides:
Index(['ash_alcanity', 'ash', 'proanthocyanins', 'color_intensity', 'alcohol',
       'nonflavanoid_phenols', 'magnesium', 'od280', 'flavanoids', 'proline',
       'total_phenols', 'hue', 'malic_acid'],
      dtype='object')


In [20]:
X = X_train.loc[:, best_important_features]

In [21]:
Ns_clusters = np.arange(2, 15, 1)

ss_list = []


# Normalização dos dados
scaler = Normalizer()
X_train_scaled = scaler.fit_transform(X)

# Redução de dimensionalidade (opcional)
pca = PCA(n_components=0.15)  # Manter 95% da variância
X_train_pca = pca.fit_transform(X_train_scaled)


for c in Ns_clusters:
    kmeans = KMeans(n_clusters=c,  
                    init='random',  # Inicialização inteligente
                    n_init=500)      # Aumentar o número máximo de iterações
    # Fit (treinamento)                   
    labels = kmeans.fit_predict(X_train_pca)

    # Performance
    ss_avg = mt.silhouette_score(X_train_pca, labels)
    ss_list.append(ss_avg)

    print('Para n_clusters = {}, a média do coeficiente de silhueta = {}'.format(c, ss_avg))



Para n_clusters = 2, a média do coeficiente de silhueta = 0.6455159805936248
Para n_clusters = 3, a média do coeficiente de silhueta = 0.5969023623345473
Para n_clusters = 4, a média do coeficiente de silhueta = 0.5728963330095751
Para n_clusters = 5, a média do coeficiente de silhueta = 0.5813438907027562
Para n_clusters = 6, a média do coeficiente de silhueta = 0.5939162915676511
Para n_clusters = 7, a média do coeficiente de silhueta = 0.5928233405886698
Para n_clusters = 8, a média do coeficiente de silhueta = 0.6097149794167036
Para n_clusters = 9, a média do coeficiente de silhueta = 0.6224354508652088
Para n_clusters = 10, a média do coeficiente de silhueta = 0.614764485527037
Para n_clusters = 11, a média do coeficiente de silhueta = 0.6148696598807972
Para n_clusters = 12, a média do coeficiente de silhueta = 0.6109711176667669
Para n_clusters = 13, a média do coeficiente de silhueta = 0.6051458295259186
Para n_clusters = 14, a média do coeficiente de silhueta = 0.597657127124

In [22]:
# Normalização dos dados
scaler = Normalizer()
X_train_scaled = scaler.fit_transform(X)

# Redução de dimensionalidade (opcional)
pca = PCA(n_components=0.15)  # Manter 95% da variância
X_train_pca = pca.fit_transform(X_train_scaled)

kmeans = KMeans(n_clusters=8,  
                init='random',  # Inicialização inteligente
                n_init=500)      # Aumentar o número máximo de iterações
    
# Fit (treinamento)                   
labels = kmeans.fit_predict(X_train_pca)

n_clustersK = len(np.unique(labels))
# Performance
Kss = mt.silhouette_score(X_train_pca, labels)
print(' a média do coeficiente de silhueta = {}'.format( Kss ))

 a média do coeficiente de silhueta = 0.6097149794167036


# silhouette_score

In [23]:


# Pré-processamento dos dados com StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Redução de dimensionalidade com PCA (encontrando o número de componentes principais adequado)
pca = PCA(n_components=0.15)
X_pca = pca.fit_transform(X_scaled)

# Ajuste do modelo Affinity Propagation com diferentes valores de damping e preference

preference_values = [-50, -40, -30, -20, -10]  # Experimente diferentes valores negativos
best_silhouette_score = -1
best_model = None

for preference in preference_values:
    af = AffinityPropagation(preference=preference).fit(X_pca)
    cluster_centers_indices = af.cluster_centers_indices_
    labels = af.labels_
    n_clusters_ = len(cluster_centers_indices)
    silhouette_score = mt.silhouette_score(X_pca, labels, metric='euclidean')

    print(f"Preference={preference}, Número de clusters={n_clusters_}, Coeficiente de silhueta={silhouette_score}")

    # Mantém o melhor modelo baseado no coeficiente de silhueta
    if silhouette_score > best_silhouette_score:
        best_silhouette_score = silhouette_score
        best_model = af

# Avaliação do melhor modelo
labels = best_model.labels_
cluster_centers_indices = best_model.cluster_centers_indices_
n_clusters_ = len(cluster_centers_indices)

print(f"\nMelhor modelo: Número de clusters={n_clusters_}, Coeficiente de silhueta={best_silhouette_score}")





/home/gustavo/.pyenv/versions/3.11.8/envs/KNN/lib/python3.11/site-packages/sklearn/cluster/_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


Preference=-50, Número de clusters=11, Coeficiente de silhueta=0.40238435585019827


/home/gustavo/.pyenv/versions/3.11.8/envs/KNN/lib/python3.11/site-packages/sklearn/cluster/_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


Preference=-40, Número de clusters=18, Coeficiente de silhueta=0.3539017651507886


/home/gustavo/.pyenv/versions/3.11.8/envs/KNN/lib/python3.11/site-packages/sklearn/cluster/_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


Preference=-30, Número de clusters=95, Coeficiente de silhueta=0.15015253594848704
Preference=-20, Número de clusters=54, Coeficiente de silhueta=0.2780742093937147
Preference=-10, Número de clusters=9, Coeficiente de silhueta=0.3844841829610872

Melhor modelo: Número de clusters=11, Coeficiente de silhueta=0.40238435585019827


/home/gustavo/.pyenv/versions/3.11.8/envs/KNN/lib/python3.11/site-packages/sklearn/cluster/_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(
/home/gustavo/.pyenv/versions/3.11.8/envs/KNN/lib/python3.11/site-packages/sklearn/cluster/_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


In [24]:
af = AffinityPropagation(preference=-40).fit(X_pca)
cluster_centers_indices = af.cluster_centers_indices_
labels = af.labels_
n_clustersA = len(cluster_centers_indices)
Ass = mt.silhouette_score(X_pca, labels, metric='euclidean')

print(f"Preference={preference}, Número de clusters={n_clustersA}, Coeficiente de silhueta={Ass}")

Preference=-10, Número de clusters=3, Coeficiente de silhueta=0.6087786640261164


/home/gustavo/.pyenv/versions/3.11.8/envs/KNN/lib/python3.11/site-packages/sklearn/cluster/_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


In [25]:
silhouette_lista = []
n_clusters_lista = []

n_clusters_lista.append(n_clustersA)
silhouette_lista.append(Kss)

n_clusters_lista.append(n_clustersK)
silhouette_lista.append(Ass)


In [26]:
algoritimos = ['kmeans' ,'AffinityPropagation'] 

resultados_tabela = pd.DataFrame({
    'algoritimos':algoritimos,
    'silhouette_score': silhouette_lista,
    'numero_clusters': n_clusters_lista
})
# Exiba a tabela de resultados
resultados_tabela

,algoritimos,silhouette_score,numero_clusters
0,kmeans,0.609715,3
1,AffinityPropagation,0.608779,8
